# Helicobacter pylori 
Используется штамм B-38.

Для выбранного генома бактерии скачать файлы разметки генома генами – feature table и сам полный геном (.fna)

Скачаем feature table и геном бактерии в формате fasta.

In [1]:
import os.path
import urllib.request

In [2]:
DATA_DIR = "./data"

if not os.path.isdir(DATA_DIR):
    os.mkdir(DATA_DIR)

def download_file_if_needed(url: str, file_name: str) -> None:
    file_path = os.path.join(DATA_DIR, file_name)
    if not os.path.isfile(file_path):
        try:
            urllib.request.urlretrieve(url, file_path)
        except e:
            print(f"Unable to download file from {url}")
            return
        if not os.path.isfile(file_path):
            print(f"Unable to save downloaded file at {file_path}")

In [3]:
# fasta файл с геномом
download_file_if_needed("https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/091/345/GCF_000091345.1_ASM9134v1/GCF_000091345.1_ASM9134v1_genomic.fna.gz", "genome.fna.gz")
# feature table
download_file_if_needed("https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/091/345/GCF_000091345.1_ASM9134v1/GCF_000091345.1_ASM9134v1_feature_table.txt.gz", "feature_table.txt.gz")

Разархивируем скачанные файлы в формате gz.

In [4]:
import gzip
import shutil

In [5]:
def unzip_data_if_needed(file_name: str) -> str:
    file_path = os.path.join(DATA_DIR, file_name)
    out_file_name = file_name[:file_name.find(".gz")]
    out_file_path = os.path.join(DATA_DIR, out_file_name)

    if os.path.isfile(out_file_path):
        return out_file_path

    with gzip.open(file_path, "rb") as file_input:
        with open(out_file_path, "wb") as file_output:
            shutil.copyfileobj(file_input, file_output)

    return out_file_path

In [6]:
# Разархивируем геном
GENOME_FILE_PATH = unzip_data_if_needed("genome.fna.gz")
# Разархивируем feature table
FEATURE_TABLE_FILE_PATH = unzip_data_if_needed("feature_table.txt.gz")

Для обработки genome.fna будем использовать библиотеку biopython, в частности её модуль SeqIO.  
Установим biopython через pip, если её еще у нас нет.

In [7]:
!pip3 install biopython

In [8]:
from Bio import SeqIO

In [9]:
# т. к. у нас только одна последовательность в genome.fna, то сразу берём её -> list(...)[0]
genome_seq = list(SeqIO.parse(GENOME_FILE_PATH, "fasta"))[0]

*Какова длина генома?*

In [10]:
print(f"Длина генома: {len(genome_seq.seq)}")

Длина генома: 1576758


Для обработки feature_table.txt будем использовать библиотеку pandas

In [11]:
!pip3 install pandas

In [12]:
import pandas as pd
import numpy as np

In [13]:
ft = pd.read_table(FEATURE_TABLE_FILE_PATH)

*Сколько генов, кодирующих белки?*

In [14]:
(ft["class"] == "protein_coding").sum()

1406

*Сколько рнк-генов?*

In [15]:
genes_ft = ft[ft["# feature"] == "gene"]
gene_classes = pd.unique(genes_ft["class"])
print(f"classes с feature = 'gene': {gene_classes}")

RNA_classes = [gene_class for gene_class in gene_classes if "RNA" in gene_class]
count = sum([(genes_ft["class"] == RNA_class).sum() for RNA_class in RNA_classes]) 
print(f"Рнк-генов: {count}")

classes с feature = 'gene': ['protein_coding' 'tRNA' 'SRP_RNA' 'pseudogene' 'tmRNA' 'RNase_P_RNA'
 'rRNA']
Рнк-генов: 45


*Сколько транскрипционных факторов?*

In [16]:
print("Транскрипционных регуляторов:")
!grep "transcriptional regulator" {FEATURE_TABLE_FILE_PATH} | wc -l

print("Транскрипционных факторов:")
!grep "transcription factor" {FEATURE_TABLE_FILE_PATH} | wc -l

Транскрипционных регуляторов:
6
Транскрипционных факторов:
3


*Сколько транспортных белков (ABC transporters)?*

In [17]:
!grep "ABC transporter" {FEATURE_TABLE_FILE_PATH} | wc -l

35


*Сколько tRNA?*

In [18]:
print((ft["# feature"] == "tRNA").sum())

36


*Сколько закодировано субъединиц АTP-synthase?*

In [19]:
!grep "ATP synthase" {FEATURE_TABLE_FILE_PATH} | wc -l

9


*Сколько генов закодировано на положительном, а сколько на отрицательном стренде?*

In [20]:
print((genes_ft["strand"] == "+").sum(), (genes_ft["strand"] == "-").sum())

695 820


# Ген человека ARID1B

*В геномном браузере UCSC отобразить все изоформы гена, а также SNPs (Common dbSNP(155) и ClinVar dbSNP(155)), участки консервативности среди позвоночных и транспозоны. Сохранить скриншот в виде графического файла.*


![ARID1B: Изоформы, SNPs, участки консервативности позвоночных, транспозоны](pics/arid1b.png)

*Отобрать в табличном виде и сохранить в текстовый файл все изоформы генов, попавших в заданный участок*

см. data/arid1b-isoforms.txt

*Отобрать координаты только экзонов и только интронов для заданного участка.*

см. data/arid1b-exons-only.tsv и data/arid1b-introns-only.tsv

*Ответить письменно, в какие участки гена попадают clinically relevant SNPs.*

На снимке сверху можно увидеть, что clinically relevant SNPs (трек ClinVar dbSNP(155) -> красный цвет) попадает только в экзоны гена.

*Ответить письменно, в какие участки гена попадают транспозоны.*

В свою очередь транспозоны попадают как в экзоны, так и в интроны гена.